# paths

> Module of path processing and utility functions


In [ ]:
# | default_exp core.paths

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export
from __future__ import annotations

import logging
import os
import re
import shutil
import weakref as _weakref
from datetime import datetime
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Iterable

from pydantic import PathNotADirectoryError

from ds_contrib.core.utils import Iterifiable, iterify

In [ ]:
# | export
# | hide

logger = logging.getLogger(__name__)

In [ ]:
# | export
PathLike = str | os.PathLike | None

We could use it to convert objects to iterables


In [ ]:
# | export
# | hide


def _list_all_paths(paths: Iterable[Path], recursive):
    """List all paths in a list of paths, if a path is a folder, list all paths in that folder.

    Parameters
    ----------
    paths : Iterable[Path]
        _description_
    recursive : _type_
        _description_

    Yields
    ------
    _type_
        _description_

    Raises
    ------
    FileNotFoundError
        _description_
    """
    for p in paths:
        if not p.exists() or p is None:
            raise FileNotFoundError(f"File or folder {p} does not exist.")
        if p.is_dir():
            if recursive:
                yield from p.rglob("*")
            else:
                yield from p.glob("*")
        else:
            yield p

In [ ]:
# | export


def list_paths(
    paths: Iterifiable[PathLike],
    pattern: re.Pattern | str | None = None,
    recursive=False,
    sort=False,
    only_files=True,
) -> Iterable[Path]:
    """List all files in paths matching a pattern.

    Parameters
    ----------
    paths : Iterifiable[PathLike]
        Single PathLike object or an iterable of PathLike objects
    pattern : re.Pattern | str | None, optional
        regex pattern to match file names, e.g. r'.*\.jpg$', r'.*\.(jpg|png)$', r'part_0000/.*\.jpg', if None, all files are returned, by default None
    recursive : bool, optional
        if True, search recursively in subfolders of paths, by default False
    sort : bool, optional
        if True, sort the resulting list of paths, by default False
    only_files : bool, optional
        if True, only return files, if False, return files and folders, by default True

    Returns
    -------
    Iterable[Path]
        Iterable of Path objects

    Raises
    ------
    ValueError
        If pattern is not a string or a compiled regex pattern or `None`.
    """

    result = _list_all_paths(
        map(lambda p: Path(p), iterify(paths)), recursive=recursive
    )

    if isinstance(pattern, (str, re.Pattern)):
        if isinstance(pattern, str):
            pattern = re.compile(pattern)
        pattern = re.compile(pattern)
    else:
        if pattern is not None:
            raise ValueError(
                f"Pattern must be a string or a compiled regex pattern or `None`, got {type(pattern)}"
            )
    if only_files:
        result = filter(lambda p: p.is_file(), result)
    if pattern:
        result = filter(lambda p: pattern.match(str(p)), result)
    return sorted(result) if sort else result

In [ ]:
list(list_paths(Path.cwd(), recursive=True))

[PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/00_core.ipynb'),
 PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/03_video.ipynb'),
 PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/02_plotting.ipynb'),
 PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/01_paths.ipynb')]

In [ ]:
list(list_paths(Path.cwd(), pattern=r".*\.(ipynb|css)$", sort=True))

[PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/00_core.ipynb'),
 PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/01_paths.ipynb'),
 PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/02_plotting.ipynb'),
 PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core/03_video.ipynb')]

In [ ]:
# | export

from requests import get

from ds_contrib.core.utils import listify


def get_dir(path: PathLike):
    path = Path(path)
    if path.is_file():
        return path.parent
    else:
        return path


def shared_root(paths: Iterifiable[PathLike], only_files=False) -> Path:
    """Find the longest common root of a list of paths.

    Parameters
    ----------
    paths : Iterable[PathLike]
        Iterable of Path objects
    only_files : bool, optional
        If True, only consider the parent directory of each path, else try to intellectually find directory from each path, by default False

    Returns
    -------
    Path
        shared root

    Raises
    ------
    ValueError
        if paths is empty
    """
    paths = listify(paths, none_handlings="empty")
    paths = list(map(lambda p: Path(p).absolute().resolve(), paths))

    if len(paths) < 1:
        raise ValueError("paths must be a non-empty list of paths")
    else:
        root = paths[0].parent if only_files else get_dir(paths[0])
        for p in paths[1:]:
            p_dir = p.parent if only_files else get_dir(p)
            if p_dir != root:
                new_root = []
                for root_part, p_part in zip(root.parts, p_dir.parts):
                    if root_part != p_part:
                        break
                    else:
                        new_root.append(root_part)
                root = Path(*new_root)

    return root

In [ ]:
shared_root([Path("data"), Path("images"), Path("cat.jpg")])

PosixPath('/Users/arseniy/Projects/dev/ds_contrib/nbs/core')

In [ ]:
# | export


def prepare_paths_for_transfer(
    paths: Iterable[PathLike],
    recursive: bool = False,
    target_root: PathLike = None,
    local_shared_root: PathLike = None,
) -> Iterable[Path]:
    """Prepare paths for transfer to a remote machine. This function will resolve all paths to absolute paths,
    and then return the paths relative to the shared root of all paths, if target_root is not None, the paths will be under target_root.

    Parameters
    ----------
    paths : Iterable[PathLike]
        iterable of paths or a single PathLike object
    recursive : bool, optional
        if True, search recursively in subfolders of paths that are directories, by default False
    target_root : PathLike, optional
        root path on the remote machine, by default None
    local_shared_root : PathLike, optional
        shared root of all paths, if None, the shared root will be computed from paths, by default None

    Returns
    -------
    Iterable[Path]
        iterator of paths
    """

    if local_shared_root is None:
        local_shared_root = shared_root(paths)
    local_shared_root = Path(local_shared_root).resolve().absolute()
    paths = list_paths(paths, recursive=recursive, only_files=True)
    paths = map(lambda p: p.resolve().absolute(), paths)
    yield from map(lambda p: Path(target_root) / p.relative_to(local_shared_root), paths)  # type: ignore

In [ ]:
list(
    prepare_paths_for_transfer(
        [Path("../nbdev.yml"), Path("../sidebar.yml")], target_root="a"
    )
)

[PosixPath('a/nbdev.yml'), PosixPath('a/sidebar.yml')]

In [ ]:
list(
    prepare_paths_for_transfer(
        [Path("../nbdev.yml"), Path("../sidebar.yml")],
        target_root="a",
        local_shared_root="../..",
    )
)

[PosixPath('a/nbs/nbdev.yml'), PosixPath('a/nbs/sidebar.yml')]

In [ ]:
# | export


class Directory:
    def __init__(self, path: PathLike = None, temporary=True):
        """Directory for storing files. Can be persistent or temporary. If temporary, it will be deleted on exit.

        Parameters
        ----------
        path : PathLike, optional
            Path to directory, if None, will be created temporary in current working directory, by default None
        temporary : bool, optional
            If True, directory will be deleted on exit, by default True
        """
        self._is_tmp = temporary

        self._path = self._init_dir(path)

        self._lazy_tmp_dir = None

        self._finalizer = _weakref.finalize(
            self, self._cleanup, self._lazy_tmp_dir, self._path
        )

    @property
    def is_initialized(self):
        return not self._is_tmp or self._lazy_tmp_dir is not None

    @property
    def path(self) -> Path:
        """Path to directory. If temporary, will be created on first call.

        Returns
        -------
        Path
            Path to directory
        """
        if not self.is_initialized:
            self._init_tmp_dir()
        return self._path

    def _init_tmp_dir(self):
        if self._path is None:
            folder = Path.cwd()
        else:
            folder = self._path
        self._lazy_tmp_dir = TemporaryDirectory(dir=folder)
        self._path = Path(self._lazy_tmp_dir.name).absolute()
        logger.info(f"Created temporary directory for downloads: `{self._path}`")

    def _init_dir(self, path: PathLike = None):
        if self._is_tmp:
            if path is None:
                logger.debug(
                    f"Temporary directory will be created with `{Path.cwd()}/$random_uuid` on first call of `path` property"
                )
            else:
                path = Path(path)
                if not path.exists():
                    raise FileNotFoundError(
                        f"Root directory for temporary directory does not exist `{path}`"
                    )
                else:
                    logger.debug(
                        f"Temporary directory will be created at `{path.absolute()}/$random_uuid` on first call of `path` property"
                    )
            return path
        else:
            path = Path(path).resolve().absolute()
            if path.is_file():
                raise PathNotADirectoryError(path=path)
            if path.exists():
                logger.info(f"Directory for downloads exists `{path}`")
                return path
            else:
                path.mkdir(parents=True, exist_ok=False)
                logger.info(f"Created persistent directory for downloads `{path}`")
                return path

    def make_persistent(self, new_path: PathLike = None):
        """Makes temporary directory persistent.
        If new_path is None, will be created in current working directory with name {dd/mm/yyyy}_{h/m/s}_tmp


        Parameters
        ----------
        new_path : PathLike, optional
            Path to new persistent directory, if exists, will raise FileExistsError, by default None

        Raises
        ------
        ValueError
            If directory is already persistent
        FileExistsError
            If new_path already exists
        """
        if not self._is_tmp:
            raise ValueError(f"Directory `{self.path}` is already persistent")

        if new_path is None:
            # create new path based on datetime format {dd/mm/yyyy}_{h/m/s}_tmp
            new_path = Path(f"{datetime.now().strftime('%d_%m_%Y-%H-%M-%S')}_tmp")
        else:
            new_path = Path(new_path)
        if new_path.exists():
            raise FileExistsError(
                f"Directory `{new_path.absolute()}` already exists, cannot move to it"
            )
        logger.info(f"Moving temporary directory `{self.path}` to `{new_path}`")
        shutil.move(self.path, new_path)
        self.cleanup()

    @classmethod
    def _cleanup(cls, tmp_dir, path: PathLike):
        if tmp_dir:
            logger.info(f"Cleaning up temporary directory `{path}`")
            tmp_dir.cleanup()

    def cleanup(self):
        """Cleanup temporary directory. If directory is persistent, will do nothing.
        Use it if you want to delete temporary directory before exit. Automatically called on deletion of the Directory object (after garbage collection)
        """
        if self._is_tmp:
            if self._lazy_tmp_dir:
                parent = self._path.parent
                self._cleanup(self._lazy_tmp_dir, self._path)
                self._lazy_tmp_dir = None
                self._path = parent
            else:
                logger.info(f"Tmp directory was not initialized, skipping cleanup")
        else:
            logger.info(f"Directory `{self._path}` is persistent, skipping cleanup")

In [ ]:
folder = Directory()
# some method using folder.path
folder.path

folder.cleanup()

In [ ]:
folder = Directory()
# some method using folder.path
folder.path

del folder

In [ ]:
folder = Directory()
folder.make_persistent("./b")
Path("./b").rmdir()

---


#| hide

Utility section


In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()